In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [196]:
pd.set_option('future.no_silent_downcasting', True)

#### Getting the data

In [7]:
symbol = input("Enter the symbol you want to perform DCF on: ")

In [8]:
ticker = yf.Ticker(symbol)

In [6]:
selected_case = int(input("""
Choose 3 cases:
Case 1: Optimistic case
Case 2: Base case
Case 3: worst case
Enter your choice (1-3): """))

In [9]:
Balance_sheet = ticker.balance_sheet.transpose()
Income_statement = ticker.income_stmt.transpose()

#### P&L assumption

##### Modelling revenues

In [299]:
actual = pd.DataFrame(columns=sorted(list(Income_statement.index.year)))

In [300]:
forecast = pd.DataFrame(columns=[x + max(list(Income_statement.index.year)) for x in range(1,6)]).transpose()

In [301]:
revenues = pd.DataFrame(Income_statement['Operating Revenue']).rename({'Operating Revenue' : 'Revenues'},axis=1)
revenues.index = revenues.index.year
revenues = revenues.transpose()

In [302]:
actual = pd.concat([actual, revenues])
actual = actual.transpose()

In [303]:
actual['y-o-y growth'] = actual['Revenues'].pct_change()
actual

,Revenues,y-o-y growth
2020,143015000000.0,NaN
2021,168088000000.0,0.175317
2022,198270000000.0,0.179561
2023,211915000000.0,0.06882


In [304]:
if selected_case == 1:
    forecast['y-o-y growth'] = max(actual['y-o-y growth'])
elif selected_case == 2:
    forecast['y-o-y growth'] = (actual['y-o-y growth']).mean()
elif selected_case == 3:
    forecast['y-o-y growth'] = (actual['y-o-y growth'].mean()) / 2

In [305]:
forecast_revenue = forecast.copy()
forecast_revenue['forecast Revenue'] = actual['Revenues'].iloc[-1] * (1 + forecast_revenue['y-o-y growth'])

In [306]:
for i in range(1, len(forecast_revenue)):
    forecast_revenue.loc[forecast_revenue.index.min() + i,'forecast Revenue'] = forecast_revenue.loc[forecast_revenue.index.min() + i -1, 'forecast Revenue'] * (1 + forecast_revenue['y-o-y growth'].iloc[0])
forecast_revenue

,y-o-y growth,forecast Revenue
2024,0.070616,2.268797e+11
2025,0.070616,2.429011e+11
2026,0.070616,2.600539e+11
2027,0.070616,2.784180e+11
2028,0.070616,2.980788e+11
